In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge

# Data Preprocessing

Load Excel file

In [2]:
xl = pd.ExcelFile("CONFIDENTIAL_Ecotagious_AlgoTestTrainDataSet_20180811.xlsx")

Create 4 dataframe for 4 sheets in the excel file

In [3]:
without_target = xl.parse("AggregateLoadWithoutTargetLoad")
without_target.columns = ["Home_ID"] + list(without_target.columns[1:])
target_load = xl.parse("TargetLoad")
target_load.columns = ["Home_ID"] + list(target_load.columns[1:])
aggregate_all = xl.parse("AggregateLoadWithTargetLoad")
aggregate_all.columns = ["Home_ID"] + list(aggregate_all.columns[1:])
results_all = xl.parse("Results")
results_all.columns = ["Home_ID"] + list(results_all.columns[1:])

In [4]:
aggregate_all.head()

,Home_ID,2016010100,2016010101,2016010102,2016010103,2016010104,2016010105,2016010106,2016010107,2016010108,...,2016123114,2016123115,2016123116,2016123117,2016123118,2016123119,2016123120,2016123121,2016123122,2016123123
0,1,0.55,0.52,0.43,0.43,1.04,0.56,0.53,0.48,0.55,...,3.52,1.36,0.44,0.42,0.82,0.45,0.44,0.43,0.53,0.42
1,2,0.25,0.31,0.31,0.27,0.27,0.33,0.30,0.24,0.32,...,0.25,0.26,0.33,0.34,0.31,0.33,0.34,0.35,0.29,0.29
2,3,3.65,0.90,0.92,0.54,0.55,0.87,0.67,0.54,1.04,...,3.74,1.15,0.67,0.73,1.00,0.56,1.02,0.72,0.78,1.06
3,4,0.66,0.70,1.13,0.46,0.73,0.65,0.46,0.78,0.81,...,4.47,2.27,5.11,3.05,0.74,0.95,1.31,0.73,0.53,0.46
4,5,0.25,0.19,0.25,0.39,0.28,0.71,0.60,0.20,0.29,...,0.22,0.30,0.20,0.29,0.25,0.34,0.30,0.26,0.19,0.29


In [5]:
results_all.head()

,Home_ID,1,2,3,4,5,6,7,8,9,10,11,12
0,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,228.689071,236.312040,236.312040,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,315.565724,326.084582,326.084582,0.0,0.0,0.0,0.0


First, we drop the *Home_ID* column of *aggregate_all* then turn its columns header into datetime object.

In [7]:
aggregate_all.drop("Home_ID", inplace=True, axis=1)
aggregate_all.columns = pd.to_datetime(aggregate_all.columns, format="%Y%m%d%H")

In [8]:
aggregate_all.head()

,2016-01-01 00:00:00,2016-01-01 01:00:00,2016-01-01 02:00:00,2016-01-01 03:00:00,2016-01-01 04:00:00,2016-01-01 05:00:00,2016-01-01 06:00:00,2016-01-01 07:00:00,2016-01-01 08:00:00,2016-01-01 09:00:00,...,2016-12-31 14:00:00,2016-12-31 15:00:00,2016-12-31 16:00:00,2016-12-31 17:00:00,2016-12-31 18:00:00,2016-12-31 19:00:00,2016-12-31 20:00:00,2016-12-31 21:00:00,2016-12-31 22:00:00,2016-12-31 23:00:00
0,0.55,0.52,0.43,0.43,1.04,0.56,0.53,0.48,0.55,0.64,...,3.52,1.36,0.44,0.42,0.82,0.45,0.44,0.43,0.53,0.42
1,0.25,0.31,0.31,0.27,0.27,0.33,0.30,0.24,0.32,0.23,...,0.25,0.26,0.33,0.34,0.31,0.33,0.34,0.35,0.29,0.29
2,3.65,0.90,0.92,0.54,0.55,0.87,0.67,0.54,1.04,0.76,...,3.74,1.15,0.67,0.73,1.00,0.56,1.02,0.72,0.78,1.06
3,0.66,0.70,1.13,0.46,0.73,0.65,0.46,0.78,0.81,0.39,...,4.47,2.27,5.11,3.05,0.74,0.95,1.31,0.73,0.53,0.46
4,0.25,0.19,0.25,0.39,0.28,0.71,0.60,0.20,0.29,0.27,...,0.22,0.30,0.20,0.29,0.25,0.34,0.30,0.26,0.19,0.29


Next, we create a new dataframe that aggregates data of *aggregrate_all* monthly.

In [9]:
aggregate_month = aggregate_all.transpose().resample("M",).sum().transpose()
aggregate_month.columns = [str(time.month) for time in aggregate_month.columns]

In [10]:
aggregate_month.head()

,1,2,3,4,5,6,7,8,9,10,11,12
0,962.29,868.05,943.89,800.44,748.47,883.780000,1056.910000,1026.340000,786.57,799.88,820.25,1018.03
1,553.57,480.86,649.38,467.80,319.03,605.889071,770.262040,697.972040,1039.33,786.06,641.75,445.61
2,1324.65,1257.77,1163.21,982.75,929.81,790.160000,842.740000,815.720000,770.70,865.96,859.27,943.95
3,1152.11,1096.92,1150.11,1242.16,1473.83,1876.820000,2360.900000,2181.050000,1664.11,1035.94,986.88,1165.57
4,295.45,274.01,253.48,234.62,208.87,516.735724,607.514582,551.814582,210.38,237.52,252.91,286.30


# Predicting Target Load

In [11]:
results_all.head()

,Home_ID,1,2,3,4,5,6,7,8,9,10,11,12
0,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,228.689071,236.312040,236.312040,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,315.565724,326.084582,326.084582,0.0,0.0,0.0,0.0


Observing *results_all* we notice that among first 60 houses (trainning set), there are houses that do not have the appliance we want to predict it load. Thus, we first build a binary classification model to detect which house has the target appliance.

Using *results_all* we create a vector that store the labels of trainning set where '1' denotes that the house has the appliance.

In [12]:
labels = results_all.drop("Home_ID", axis=1).iloc[:60].any(axis=1).astype("int")

In [13]:
labels[:5]

0    0
1    1
2    0
3    0
4    1
dtype: int64

## Binary Classification

Since we observe that the appliance only operates in June, July, and August, our approach is to build the classifier based on the trainning data of June, July, and August (hourly data).

In [15]:
# Trainning features
train_feature = aggregate_all.transpose()["2016-06":"2016-08"].transpose()[:60]

# Classifier
svc = LinearSVC()

# Fit to the trainning data
svc.fit(train_feature, labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [16]:
# Cross validation scores accuracy of the classfier
print(np.average(cross_val_score(svc, train_feature, labels)))

0.933166248956


In [17]:
# Testing features
test_feature = aggregate_all.transpose()["2016-06":"2016-08"].transpose()[60:]

# Predict labels for test set
has_app_pred = svc.predict(test_feature)

In [18]:
# has_app_pred is the prediction labels of 50 houses in the testing set.
has_app_pred

array([0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0])

In [19]:
# index of the house in the test set that is predicted having target appliance
has_app_test = [item for i, item in enumerate(aggregate_all[60:].index.values) if has_app_pred[i] == 1]

## Regression

Next, our objective is predicting the target load of the house that has the target appliance. We build a regression model on a subset of the tranning set. The subset contains only houses that has label "1". The labels of the trainning set is the target loads in *result_all* for those rows that are labeled "1".

In [20]:
# index of the house in the trainning set that has target appliance
has_app = [index for index, v in enumerate(labels) if v==1 ]

# Trainning set of the regression model
train_reg = aggregate_all.transpose()["2016-06":"2016-08"].transpose().loc[has_app]

# Ridge Regression
reg = Ridge(alpha=0.1)

# Trainning label
train_label = results_all.iloc[has_app].loc[:,[6,7,8]]

# Fit to the trainning data
reg.fit(train_reg, train_label)

# Testing feature
test_reg = aggregate_all.transpose()["2016-06":"2016-08"].transpose().loc[has_app_test]

# Prediction labels of test set
test_pred = reg.predict(test_reg)

In [21]:
test_pred

array([[ 290.47256154,  300.15498025,  300.15498025],
       [ 237.59804397,  245.51797877,  245.51797877],
       [ 276.85934172,  286.08798645,  286.08798645],
       [ 248.11369538,  256.3841519 ,  256.3841519 ],
       [ 240.15649933,  248.16171597,  248.16171597],
       [ 235.90901874,  243.7726527 ,  243.7726527 ],
       [ 302.37506036,  312.45422904,  312.45422904],
       [ 225.71602115,  233.23988852,  233.23988852],
       [ 254.02884085,  262.49646888,  262.49646888],
       [ 263.0002069 ,  271.76688046,  271.76688046],
       [ 261.91992202,  270.65058608,  270.65058608],
       [ 270.83390706,  279.86170396,  279.86170396],
       [ 221.75569767,  229.14755426,  229.14755426],
       [ 107.93646472,  111.53434688,  111.53434688],
       [ 194.38216946,  200.8615751 ,  200.8615751 ],
       [ 283.05914043,  292.49444511,  292.49444511],
       [ 217.67233775,  224.92808234,  224.92808234],
       [ 212.72096893,  219.81166789,  219.81166789],
       [ 241.99655393,  250.

# Results

In [22]:
results_final = results_all.copy()
results_final.loc[60:,:] = 0

In [23]:
results_final.loc[has_app_test,[6,7,8]] = test_pred

In [24]:
results_final

,Home_ID,1,2,3,4,5,6,7,8,9,10,11,12
0,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,228.689071,236.312040,236.312040,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,315.565724,326.084582,326.084582,0.0,0.0,0.0,0.0
5,6,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
6,7,0.0,0.0,0.0,0.0,0.0,221.144025,228.515492,228.515492,0.0,0.0,0.0,0.0
7,8,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
8,9,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
9,10,0.0,0.0,0.0,0.0,0.0,227.977524,235.576775,235.576775,0.0,0.0,0.0,0.0


# Output to Excel file

In [31]:
results_final.to_excel('./output.xlsx')